# Author prediction using RNNs

In this notebook, you will prepare a dataset to predict authorship of the Federalist Papers using Recurrent Neural Networks (RNNs).

## Federalist Papers

The Federalist Papers are a collection of 85 articles and essays written by Alexander Hamilton, James Madison, and John Jay under the pseudonym "Publius". They were published in 1787 and 1788 to promote the ratification of the United States Constitution. The authors of the Federalist Papers wanted to remain anonymous, so they used the pseudonym "Publius". The articles were published in two newspapers, the New York Packet and the Independent Journal. The Federalist Papers are considered important works of American political thought and are still widely read today.

## Additional information

- [Federalist Papers](https://en.wikipedia.org/wiki/The_Federalist_Papers)
- [Federalist Papers dataset](https://www.kaggle.com/datasets/tobyanderson/federalist-papers)
- [LOC Research Guide](https://guides.loc.gov/federalist-papers/full-text)

## Objectives (a)

- Task 1: Design your dataset to predict the author of a text using RNNs.
- Task 2: Explain the methodology of your dataset creation.
- Task 3: Establish a baseline model using Machine Learning algorithms (SVM, Random Forest, etc.).
  
## Objectives (b)

- Task 4: Train a RNN model to predict the author of a text.
- Task 5: Evaluate the performance of your model.
- Task 6: Explain your model architecture, hyperparameters, and the results of your model.

## Task 1: Dataset Creation

### Dataset description

Use the below code to scrape the Library of Congress research guide for the Federalist Papers. You need to pip install the following for the below code to work:

- `pip install beautifulsoup4`
- `pip install requests`
- `pip install pandas`

In [ ]:
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup as bs
import requests

url = 'https://guides.loc.gov/federalist-papers/full-text'

html = requests.get(url).text
soup = bs(html, 'html.parser')

# convert the html table to a pandas dataframe
table = soup.find('table')

# List of Federalist Papers
meta_df = pd.read_html(StringIO(str(table)), parse_dates=True)[0]

### Dataset datatypes

In [ ]:
meta_df.info()

In [ ]:
meta_df.head()

### Autorship counts

Plot the count the number of papers written by each author. We want to create a machine learning dataset that consists of a train, validation, and test set. We will use the train set to train the model, the validation set to tune the hyperparameters, and the test set to evaluate the model.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# plot the number of papers written by each author
plt.figure(figsize=(10, 6))
sns.countplot(y='Author', data=meta_df)
plt.title('Number of Papers Written by Each Author')
plt.show()

Based on the data, answer the following questions:

- How many papers were written by each author?
- How should you construct your dataset? Explain your reasoning.

### Contested authorship

In [ ]:
# disputed papers
disputed_papers = meta_df[meta_df['Author'] == 'Hamilton or Madison']
contested_authorship = disputed_papers['No.'].values
print(f'Contested authorship of papers: {contested_authorship}')
print(f'Total number of disputed papers: {len(disputed_papers)}')

### Load the Federalist Papers corpus

The following code loads the Federalist Papers corpus. The papers have been preprocessed to aid your analysis of the authorship of the papers.

In [ ]:
corpus = pd.read_pickle('../datasets/Federalist_Papers/fp_corpus.pkl')
corpus.head()

The above datasets contains the following columns:

- `paper_id`: The ID of the paper. This corresponds to the number of the paper in the Federalist Papers collection.
- `author`: The author of the paper. This can be one of three values: `Hamilton`, `Madison`, `Jay`, `dispt` (disputed), and `HM` (Hamilton and Madison).
- `text`: The text of the paper.
- `sentence_length`: The token count of the sentence, defined as the number of tokens by splitting the sentence by spaces.
- `sentence_index`: The index of the sentence in the paper.
- `total_sentences`: The total number of sentences in the paper.

The sentences were created by using the Spacy transformer library to split the text of papers into sentences.

In [ ]:
# drop the duplicates and keep target and paper_id
authorship = corpus.drop_duplicates(subset=['paper_id', 'target'])
authorship.target.value_counts()

### Dataset creation

Your task is to predict the author of a paper using the text of the paper. You will use a Recurrent Neural Network (RNN) to predict the author of a paper. You will use the Federalist Papers dataset to train and evaluate your model.

In [ ]:
# YOUR CODE HERE - CREATE A DATASET FOR THE CLASSIFICATION TASK OF AUTHORSHIP ATTRIBUTION

### Plot the distribution of the number of papers written by each author.

In [ ]:
# YOUR CODE HERE - Visualize the distribution of the sentence lengths (what is the quartile distribution of the sentence lengths?)


### Plot the distribution of sentence lengths for each author.

In [ ]:
# YOUR CODE HERE - Visualize the distribution of sentence lengths according to the authorship

### Prepare dataset for training

Prepare a training, validation, and test dataset. What is your X and y? What is the size of your training, validation, and test dataset?

In [ ]:
from sklearn.model_selection import train_test_split

X_train, x_val, y_train, y_val = # YOUR CODE HERE
print(f'Training samples: {len(X_train)}')
print(f'Validation samples: {len(x_val)}')

#### Visualize some random samples

In [ ]:
import random

# sample a random sentence from the training set
random_idx = random.randint(0, len(X_train))
print(f'Random sentence: {X_train.iloc[random_idx]}')
print(f'Author: {y_train.iloc[random_idx]}')

#### Plot the distribution of training and validation samples

Have you split the dataset in a stratified manner? Explain why it is important to split the dataset in a stratified manner. If you have not, explain why you did not split the dataset in a stratified manner. See the documentation for [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) for more information.

#### Plot the datasets

In [ ]:
# plot the distribution of the target classes in the training and validation set on a countplot
fig, ax = plt.subplots(1, 2, figsize=(15, 6))
# first plot is the training set
sns.countplot(y_train, ax=ax[0])
ax[0].set_title('Training Set Distribution')
# second plot is the validation set
sns.countplot(y_val, ax=ax[1])
ax[1].set_title('Validation Set Distribution')
plt.show()

#### Convert the target labels to one hot encodings

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)

y_train_ohe = # YOUR CODE HERE
y_val_ohe = # YOUR CODE HERE

print(f'One hot encoded training labels shape: {y_train_ohe.shape}')
print(f'One hot encoded validation labels shape: {y_val_ohe.shape}')

In [ ]:
y_train_ohe[:5], y_train[:5]

#### Encode the labels

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_le = # YOUR CODE HERE
y_val_le = # YOUR CODE HERE

print(f'Label encoded training labels shape: {y_train_le.shape}')
print(f'Label encoded validation labels shape: {y_val_le.shape}')

In [ ]:
for i in range(5):
    print(f'Original label: {y_train.iloc[i]} - Label encoded: {y_train_le[i]}')

In [ ]:
num_classes = len(label_encoder.classes_)
class_names = label_encoder.classes_
num_classes, class_names

## Train a TF-IDF model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [ ]:
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', SVC(kernel='linear', C=1.0, random_state=42))
])

# fit the model
text_clf.fit(X_train, y_train_le)

In [ ]:
text_clf.score(x_val, y_val_le)

### Explain the TF-IDF model

- What is the TF-IDF model?
- How does the TF-IDF model work?
- What are the hyperparameters of the TF-IDF model?
- How do you select the hyperparameters of the TF-IDF model?
- Why is it helpful to establish a baseline model?

# Task 2: Train a Recurrent Neural Network (RNN) model

In the second part of this assignment, you are tasked with the following:

- Train a Recurrent Neural Network (RNN) model to predict the author of a paper.
- Evaluate the performance of your model and compare it against the baseline model.
- Explain your model architecture, hyperparameters, and the results of your model.
- Discuss the challenges you faced during the model training process and how you overcame them.
- Classify disputed papers using the Naive Bayes and RNN models
- Write a summary of your findings and the predictions of your model with respect to the disputed papers.

## Train a Recurrent Neural Network (RNN) model

### Vectorize your text data

Vectorize your text data with `tensorflow.keras.models.TextVectorization`. See the documentation here: [TextVectorization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization).

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import TextVectorization

# YOUR CODE HERE - Create a TextVectorization layer - what arguments will you use? Explain your choice of arguments.

text_vectorizer = TextVectorization()

In [ ]:
# Randomly visualize some of your vectorized textual data
import random
example_sent = random.choice(X_train.values) # change var name if you need to
print(f'Original text:\n{example_sent}')
print(f'\nVectorized text:\n{text_vectorizer([example_sent])}')
print('Length of vector:', len(text_vectorizer([example_sent]).numpy()[0]))

In [ ]:
# examine the vocabulary
vocab = text_vectorizer.get_vocabulary()
print(f'Number of words in the vocabulary: {len(vocab)}')
print(f'Most common words in the vocabulary: {vocab[:5]}')
print(f'Least common words in the vocabulary: {vocab[-5:]}')

In [ ]:
# examine the config vars
text_vectorizer.get_config() # when you train your model, these parameters can be changed to perhaps improve performance

In [ ]:
# Adapt the text vectorizer to the training data
text_vectorizer.adapt(YOUR_TRAINING_DATA)

### Create an embedding layer

In [ ]:
token_embedding = tf.keras.layers.Embedding(input_dim=len(vocab),
                                            output_dim=128,
                                            mask_zero=True,
                                            name='token_embedding')

print(f'Sentence before vectorization: {example_sent}')
vectorized_sent = text_vectorizer(example_sent)
print(f'Sentence after vectorization: {vectorized_sent}')
embedded_sent = token_embedding(vectorized_sent)
print(f'Sentence after embedding: {embedded_sent}')

### Create Tensorflow Dataset

To efficiently train your model, you should create a `tf.data.Dataset` object. See the documentation here: [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

In [ ]:
X_dataset = tf.data.Dataset.from_tensor_slices((YOUR CODE HERE))
val_dataset = tf.data.Dataset.from_tensor_slices((YOUR CODE HERE))

In [ ]:
# print the first 5 samples
for sample in X_dataset.take(5):
    sentence, label = sample
    print(f'Sentence: {sentence} - Label: {label}')

### Create batches of data

Batch size is a model hyperparameter that defines the number of samples that will be propagated through the network. This number can and should be adjusted in consideration with model performance and compute memory. See the documentation here: [Batch size](https://www.tensorflow.org/guide/data).

In [ ]:
BATCH_SIZE = #YOUR CODE HERE

In [ ]:
train_dataset = X_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

QUESTION: What does tf.data.AUTOTUNE do?

### Design your model

In [ ]:
# MODEL Hyperparameters
learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = 'categorical_crossentropy'
epochs = 8

In [ ]:
# Model architecture
from tensorflow.keras import layers, regularizers

inputs = tf.keras.layers.Inputs(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
# YOUR ADDITIONAL LAYERS HERE
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# Train the model or fit the model on the data
history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

QUESTION: How can tensorflow automatically save the best performing model for you?

## Evaluate the performance of your model

In [ ]:
# plot the training history
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    plt.show()

In [ ]:
plot_history(history)

- QUESTION: Is your model overfit? Underfit?
- QUESTION: Explain how bias and variance relate to training your model?

## Explain your model architecture, hyperparameters, and the results of your model

- QUESTION: What hyperparameters resulted in the best performing model?
- QUESTION: What parameters resulted in the best performing model?
- QUESTION: What model architecture resulted in the best performing model?

## Challenges faced during the model training process

Please explain any challenges you faced and how you overcame them.

## Classify the disputed authorship data

Using the Naive Bayes and RNN models, classify the disputed author sentences. Use visuaulizations and confusion matrices to communicate the decision outputs of your models. Are there any trends?

In [2]:
# YOUR CODE HERE

## Summary of your findings and predictions

- Classify the undetermined sentences with the Naive Bayes Model and your RNN model. Compare and contrast the results.
- What are your conclusions based on your experiments?
- Describe how a data science NLP solution relates to other methdologies for addressing author identification. Can we and should we value the RNN outputs? How does our RNN solution relate to issues of human values, judgments, and biases of opinion?

## Extra credit

Go back to your dataset creation and randomly change the author of the assigned sentence. With the labels randomized, rerun the code above. Describe what you learned in this process. (2pts to final grade)